<a href="https://colab.research.google.com/github/chirag-79/hms/blob/main/CNN_%2B_RF_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Step 1: Load Dataset

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import zipfile
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define paths
DATA_CSV = "/content/train.csv"  # CSV containing 'id_code' and 'diagnosis'
ZIP_PATH = "/content/drive/MyDrive/train_images.zip"
IMAGE_FOLDER = "/content/train_images"

# Extract images only if they are not already extracted
if not os.path.exists(IMAGE_FOLDER):
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(IMAGE_FOLDER)

# Load dataset
full_df = pd.read_csv(DATA_CSV)

# Ensure correct file format
full_df["id_code"] = full_df["id_code"].astype(str) + ".png"
labels = full_df["diagnosis"]

# Encode labels
unique_labels = sorted(labels.unique())  # Unique clsses
label_to_index = {label: idx for idx, label in enumerate(unique_labels)}
labels_encoded = np.array([label_to_index[label] for label in labels])

# Split dataset: Train (70%), Validation (15%), Test (15%)
train_ids, temp_ids, y_train, y_temp = train_test_split(full_df["id_code"], labels_encoded, test_size=0.3, random_state=42)
val_ids, test_ids, y_val, y_test = train_test_split(temp_ids, y_temp, test_size=0.5, random_state=42)

# Image size
IMG_HEIGHT, IMG_WIDTH = 128, 128  # Adjust based on dataset

# Function to load images, skipping missing ones
def load_images(image_ids, image_folder):
    images, valid_labels = [], []
    for img_id in image_ids:
        img_path = os.path.join(image_folder, img_id)
        if os.path.exists(img_path):  # Only process existing images
            img = load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))  # Resize
            img = img_to_array(img) / 255.0  # Normalize
            images.append(img)
        else:
            print(f"Warning: Image {img_path} not found!")

    return np.array(images)

# Load images for each dataset
X_train = load_images(train_ids, IMAGE_FOLDER)
X_val = load_images(val_ids, IMAGE_FOLDER)
X_test = load_images(test_ids, IMAGE_FOLDER)

# Convert labels to one-hot encoding
num_classes = len(unique_labels)
y_train = to_categorical(y_train, num_classes=num_classes)
y_val = to_categorical(y_val, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)

# Debugging: Print dataset shapes
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")


Mounted at /content/drive


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/train_images.zip'

In [ ]:
import matplotlib.pyplot as plt
import cv2
import seaborn as sns

# Set style for better visualization
sns.set_style("white")

# Display 15 sample images from the training set
plt.figure(figsize=(15, 15))
count = 1

for i in range(15):  # Display first 15 images
    img_name = train_ids.iloc[i]  # Get image filename
    diagnosis = y_train[i].argmax()  # Convert one-hot encoded label back to class index

    img_path = os.path.join(IMAGE_FOLDER, img_name)

    if os.path.exists(img_path):
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

        plt.subplot(5, 5, count)
        plt.imshow(img)
        plt.title(f"ID: {img_name}\nDiagnosis: {diagnosis}")
        plt.axis("off")
        count += 1
    else:
        print(f"Warning: Image {img_path} not found!")

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set style for better visualization
sns.set_style("white")

def visualize_samples(image_ids, data, labels, title, num_samples=15):
    plt.figure(figsize=(15, 15))
    count = 1

    for i in range(min(num_samples, len(data))):
        img = data[i]  # Get the image
        img = (img * 255).astype("uint8")  # Convert back to 8-bit format

        label = labels[i].argmax()  # Convert one-hot encoded label back to class index
        img_id = image_ids.iloc[i]  # Get corresponding image ID

        plt.subplot(5, 5, count)
        plt.imshow(img)  # Show correctly formatted image
        plt.title(f"ID: {img_id}\nDiagnosis: {label}", fontsize=10)
        plt.axis("off")
        count += 1

    plt.suptitle(title, fontsize=16)
    plt.show()

# Visualize validation set
visualize_samples(val_ids, X_val, y_val, "Validation Set Samples")

# Visualize test set
visualize_samples(test_ids, X_test, y_test, "Test Set Samples")


# Train CNN

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.optimizers import Adam

def create_cnn(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    x = Conv2D(32, (3, 3), padding='same', activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Conv2D(32, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)

    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)

    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)

    x = Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(256, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)

    # Uncomment below to include the 512 layer block
    # x = Conv2D(512, (3, 3), padding='same', activation='relu')(x)
    # x = BatchNormalization()(x)
    # x = Conv2D(512, (3, 3), activation='relu')(x)
    # x = BatchNormalization()(x)
    # x = MaxPooling2D(pool_size=(2, 2))(x)
    # x = Dropout(0.25)(x)

    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)

    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


# Create CNN model
cnn_model = create_cnn((IMG_HEIGHT, IMG_WIDTH, 3), len(unique_labels))

# Train CNN model
cnn_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),  # Use the predefined validation set
    epochs=100,
    batch_size=64
)

# Evaluate on test set
test_loss, test_acc = cnn_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc * 100:.2f}%")


In [ ]:
cnn_model.build((None, IMG_HEIGHT, IMG_WIDTH, 3))  # Ensure the model is built
cnn_model.summary()  # Check the architecture


# Extract Features Using CNN

In [ ]:
# Ensure the model is built and initialized
cnn_model.predict(np.random.rand(1, IMG_HEIGHT, IMG_WIDTH, 3))


In [ ]:
import numpy as np
from tensorflow.keras.models import Model

# Dummy data to run a forward pass and initialize the model
dummy_input = np.random.rand(1, IMG_HEIGHT, IMG_WIDTH, 3).astype(np.float32)
_ = cnn_model(dummy_input)  # This line initializes the model

# Extract convolutional layers
conv_layers = [layer for layer in cnn_model.layers if isinstance(layer, Conv2D)]

# Now create feature extractor models
feature_extractors = [Model(inputs=cnn_model.input, outputs=layer.output) for layer in conv_layers] # Now cnn_model.input is defined

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np # Import numpy

# Convert labels to integers
y_train_labels = np.argmax(y_train, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

# Store results
layer_accuracies = []

# Define a smaller batch size for feature extraction
batch_size = 64 # You can adjust this based on your GPU memory

for i, extractor in enumerate(feature_extractors):
    print(f"\n🔍 Extracting features from Conv Layer {i+1}/{len(feature_extractors)}")

    # Extract features in batches
    X_train_f = extractor.predict(X_train, batch_size=batch_size)
    X_test_f = extractor.predict(X_test, batch_size=batch_size)


    # Flatten
    X_train_f = X_train_f.reshape(X_train_f.shape[0], -1)
    X_test_f = X_test_f.reshape(X_test_f.shape[0], -1)

    # Train Random Forest
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_model.fit(X_train_f, y_train_labels)

    # Predict
    preds = rf_model.predict(X_test_f)

    # Evaluate
    acc = accuracy_score(y_test_labels, preds)
    print(f"✅ Accuracy using Conv Layer {i+1}: {acc:.4f}")
    layer_accuracies.append(acc)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Confusion matrix
cm = confusion_matrix(y_test_labels, preds)

# Display confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap="Blues", values_format='d')
plt.title(f"Confusion Matrix - Conv Layer {i+1}")
plt.show()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 4))
plt.plot(range(1, len(layer_accuracies) + 1), layer_accuracies, marker='o')
plt.xticks(range(1, len(layer_accuracies) + 1))
plt.xlabel("Conv2D Layer Number")
plt.ylabel("Accuracy")
plt.title("Random Forest Accuracy vs. CNN Layer Features")
plt.grid(True)
plt.show()

In [ ]:
from tensorflow.keras.models import Model
import numpy as np
import matplotlib.pyplot as plt

# Ensure the model is initialized
cnn_model.predict(np.random.rand(1, IMG_HEIGHT, IMG_WIDTH, 3))  # Dummy input to initialize

# Get all convolutional layers
conv_layers = [layer for layer in cnn_model.layers if isinstance(layer, Conv2D)]

# Create feature extractor models
feature_extractors = [Model(inputs=cnn_model.input, outputs=layer.output) for layer in conv_layers]

# Select a sample image (first image in X_train)
sample_image = np.expand_dims(X_train[0], axis=0)  # Add batch dimension

# Extract feature maps from all convolutional layers
feature_maps_list = [extractor.predict(sample_image) for extractor in feature_extractors]

# Function to plot feature maps
def plot_all_feature_maps(feature_maps_list, conv_layers):
    for feature_maps, layer in zip(feature_maps_list, conv_layers):
        num_features = feature_maps.shape[-1]  # Number of feature maps (channels)
        fig, axes = plt.subplots(1, min(8, num_features), figsize=(20, 20))
        fig.suptitle(f'Feature Maps from Layer: {layer.name}', fontsize=16)

        for i in range(min(8, num_features)):  # Show up to 8 feature maps
            ax = axes[i]
            ax.imshow(feature_maps[0, :, :, i], cmap='viridis')
            ax.axis('off')

        plt.show()

# Visualize feature maps
plot_all_feature_maps(feature_maps_list, conv_layers)


In [ ]:
from tensorflow.keras.models import Model

# Find the last fully connected dense layer before softmax
feature_layer = None
for layer in reversed(cnn_model.layers):
    if isinstance(layer, Dense):  # Get last dense layer before softmax
        feature_layer = layer
        break

# Create feature extractor model (excluding final softmax layer)
feature_extractor = Model(inputs=cnn_model.input, outputs=feature_layer.output)

# Extract features from train and test data
X_train_features = feature_extractor.predict(X_train)
X_test_features = feature_extractor.predict(X_test)

# Convert labels back to integer format for Random Forest
y_train_rf = np.argmax(y_train, axis=1)
y_test_rf = np.argmax(y_test, axis=1)


# Train Random Forest on Extracted Features

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import numpy as np

# Predict
y_pred_rf = rf_model.predict(X_test_features)

# Confusion matrix
cm = confusion_matrix(y_test_rf, y_pred_rf)

# Initialize sensitivity and specificity lists
sensitivities = []
specificities = []

for i in range(len(cm)):
    TP = cm[i, i]
    FN = np.sum(cm[i, :]) - TP
    FP = np.sum(cm[:, i]) - TP
    TN = np.sum(cm) - (TP + FN + FP)

    sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0
    specificity = TN / (TN + FP) if (TN + FP) > 0 else 0

    sensitivities.append(sensitivity)
    specificities.append(specificity)

# Accuracy
accuracy = accuracy_score(y_test_rf, y_pred_rf)

# Print results
print(f"Random Forest Accuracy: {accuracy:.4f}")
for idx, (sens, spec) in enumerate(zip(sensitivities, specificities)):
    print(f"Class {idx}: Sensitivity = {sens:.4f}, Specificity = {spec:.4f}")


#  Visualize Filters (Kernels) Learned by CNN

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Get first Conv2D layer filters
for layer in cnn_model.layers:
    if "conv" in layer.name:
        filters, biases = layer.get_weights()
        break

# Normalize filter values to 0-1 for better display
f_min, f_max = filters.min(), filters.max()
filters = (filters - f_min) / (f_max - f_min)

num_filters = filters.shape[3]
num_channels = filters.shape[2]

# Plot
fig, axes = plt.subplots(nrows=4, ncols=8, figsize=(12, 6))
axes = axes.flatten()

for i in range(min(num_filters, 32)):  # Show up to 32 filters
    ax = axes[i]
    # Combine all channels as RGB if 3 channels
    if num_channels == 3:
        ax.imshow(filters[:, :, :, i])
    else:
        ax.imshow(filters[:, :, 0, i], cmap='gray')  # fallback to grayscale
    ax.axis("off")

plt.suptitle("Filters from First Conv2D Layer", fontsize=14)
plt.tight_layout()
plt.show()


# Visualize Feature Maps (Activations)

In [ ]:
from tensorflow.keras.models import Model

# Pick an image from test set
sample_image = X_test[0]  # Pick first test image
sample_image = np.expand_dims(sample_image, axis=0)  # Add batch dimension

# Get a specific layer's output
layer_outputs = [layer.output for layer in cnn_model.layers if "conv" in layer.name]
visualization_model = Model(inputs=cnn_model.input, outputs=layer_outputs)

# Get feature maps
feature_maps = visualization_model.predict(sample_image)

# Plot feature maps of the first Conv2D layer
fig, axes = plt.subplots(nrows=4, ncols=8, figsize=(12, 6))
axes = axes.flatten()

for i in range(min(len(feature_maps[0][0, :, :, :]), 32)):  # Show first 32 feature maps
    ax = axes[i]
    ax.imshow(feature_maps[0][0, :, :, i], cmap="viridis")
    ax.axis("off")

plt.tight_layout()
plt.show()


# Project Features Using PCA


In [ ]:
from sklearn.decomposition import PCA
import seaborn as sns

# Perform PCA on extracted features
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train_features)

# Scatter plot of the features
plt.figure(figsize=(8, 6))
sns.scatterplot(x=X_train_pca[:, 0], y=X_train_pca[:, 1], hue=y_train_rf, palette="viridis")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.title("PCA Visualization of Extracted Features")
plt.show()


# Evaluate Features with Random Forest

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test_rf, y_pred_rf))


#  Generate Class Activation Maps (CAMs)

In [ ]:
import tensorflow.keras.backend as K
import cv2

# Pick a test image
img = X_test[0]
img = np.expand_dims(img, axis=0)

# Get model predictions
preds = cnn_model.predict(img)
pred_class = np.argmax(preds[0])  # Get highest probability class

# Get last convolutional layer by name
# Instead of using a hardcoded name, find the last Conv2D layer:
last_conv_layer = None
for layer in reversed(cnn_model.layers):  # Iterate in reverse order
    if isinstance(layer, Conv2D):
        last_conv_layer = layer
        break

# If no convolutional layer was found, raise an error
if last_conv_layer is None:
    raise ValueError("No convolutional layers found in the model!")

# Get gradients of predicted class wrt feature maps
grad_model = Model(inputs=cnn_model.input, outputs=[last_conv_layer.output, cnn_model.output])
with tf.GradientTape() as tape:
    conv_output, predictions = grad_model(img)
    loss = predictions[:, pred_class]

grads = tape.gradient(loss, conv_output)
pooled_grads = K.mean(grads, axis=(0, 1, 2))

# Apply gradients to feature map
heatmap = np.mean(conv_output[0], axis=-1)
heatmap = np.maximum(heatmap, 0)  # Apply ReLU
heatmap /= np.max(heatmap)  # Normalize

# Superimpose heatmap on image
# Replace 'TEST_IMAGE_FOLDER' and 'test_ids.iloc[0]' with your actual image path
# img = cv2.imread(TEST_IMAGE_FOLDER + test_ids.iloc[0])
img = X_test[0] * 255 #Assuming X_test is normalized, convert it back to 0-255 range
img = img.astype(np.uint8) # Convert to uint8 for cv2 compatibility
img = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH))

heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
heatmap = np.uint8(255 * heatmap)
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

superimposed_img = cv2.addWeighted(img, 0.6, heatmap, 0.4, 0)
plt.imshow(superimposed_img)
plt.axis("off")
plt.show()

# Plot Accuracy and Loss

In [ ]:
import matplotlib.pyplot as plt

# Get training history
history = cnn_model.history.history

# Plot training and validation accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history['accuracy'], label='Training Accuracy')
plt.plot(history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot training and validation loss
plt.subplot(1, 2, 2)
plt.plot(history['loss'], label='Training Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import zipfile
from google.colab import drive
import matplotlib.pyplot as plt
import cv2
import seaborn as sns
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.decomposition import PCA
import tensorflow.keras.backend as K

# --- 1. Setup and Data Loading --- #
drive.mount('/content/drive')

DATA_CSV = "/content/train.csv"
ZIP_PATH = "/content/drive/MyDrive/train_images.zip"
IMAGE_FOLDER = "/content/train_images"

if not os.path.exists(IMAGE_FOLDER):
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(IMAGE_FOLDER)

full_df = pd.read_csv(DATA_CSV)
full_df["id_code"] = full_df["id_code"].astype(str) + ".png"
labels = full_df["diagnosis"]

unique_labels = sorted(labels.unique())
label_to_index = {label: idx for idx, label in enumerate(unique_labels)}
labels_encoded = np.array([label_to_index[label] for label in labels])

train_ids, temp_ids, y_train_idx, y_temp_idx = train_test_split(full_df["id_code"], labels_encoded, test_size=0.3, random_state=42)
val_ids, test_ids, y_val_idx, y_test_idx = train_test_split(temp_ids, y_temp_idx, test_size=0.5, random_state=42)

IMG_HEIGHT, IMG_WIDTH = 128, 128

def load_images(image_ids, image_folder):
    images = []
    for img_id in image_ids:
        img_path = os.path.join(image_folder, img_id)
        if os.path.exists(img_path):
            img = load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
            img = img_to_array(img) / 255.0
            images.append(img)
        else:
            print(f"Warning: Image {img_path} not found!")
    return np.array(images)

X_train = load_images(train_ids, IMAGE_FOLDER)
X_val = load_images(val_ids, IMAGE_FOLDER)
X_test = load_images(test_ids, IMAGE_FOLDER)

num_classes = len(unique_labels)
y_train = to_categorical(y_train_idx, num_classes=num_classes)
y_val = to_categorical(y_val_idx, num_classes=num_classes)
y_test = to_categorical(y_test_idx, num_classes=num_classes)

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

# --- 2. Initial Data Visualization --- #
sns.set_style("white")

plt.figure(figsize=(15, 15))
for i in range(15):
    img_name = train_ids.iloc[i]
    diagnosis = y_train[i].argmax()
    img_path = os.path.join(IMAGE_FOLDER, img_name)
    if os.path.exists(img_path):
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.subplot(5, 5, i + 1)
        plt.imshow(img)
        plt.title(f"ID: {img_name}\nDiagnosis: {diagnosis}")
        plt.axis("off")
plt.show()

def visualize_samples(image_ids, data, labels, title, num_samples=15):
    plt.figure(figsize=(15, 15))
    for i in range(min(num_samples, len(data))):
        img = (data[i] * 255).astype("uint8")
        label = labels[i].argmax()
        img_id = image_ids.iloc[i]
        plt.subplot(5, 5, i + 1)
        plt.imshow(img)
        plt.title(f"ID: {img_id}\nDiagnosis: {label}", fontsize=10)
        plt.axis("off")
    plt.suptitle(title, fontsize=16)
    plt.show()

visualize_samples(val_ids, X_val, y_val, "Validation Set Samples")
visualize_samples(test_ids, X_test, y_test, "Test Set Samples")

# --- 3. CNN Model Definition and Training --- #
def create_cnn(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), padding='same', activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Conv2D(32, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)

    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)

    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)

    x = Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(256, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)

    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

cnn_model = create_cnn((IMG_HEIGHT, IMG_WIDTH, 3), num_classes)

# Train CNN model
history = cnn_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=64,
    verbose=1
)

test_loss, test_acc = cnn_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

cnn_model.build((None, IMG_HEIGHT, IMG_WIDTH, 3))
cnn_model.summary()

# --- 4. Plot Training History --- #
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# --- 5. Feature Extraction with Random Forest on Conv Layers --- #
# Dummy input to ensure model is fully built before creating feature extractors
_ = cnn_model.predict(np.random.rand(1, IMG_HEIGHT, IMG_WIDTH, 3))

conv_layers = [layer for layer in cnn_model.layers if isinstance(layer, Conv2D)]
feature_extractors_conv = [Model(inputs=cnn_model.input, outputs=layer.output) for layer in conv_layers]

y_train_labels = np.argmax(y_train, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

layer_accuracies = []
batch_size = 64
last_conv_preds = None # To store predictions for the last conv layer for CM
last_conv_idx = 0 # To store the index for the CM title

for i, extractor in enumerate(feature_extractors_conv):
    print(f"\n🔍 Extracting features from Conv Layer {i+1}/{len(feature_extractors_conv)}")
    X_train_f = extractor.predict(X_train, batch_size=batch_size)
    X_test_f = extractor.predict(X_test, batch_size=batch_size)

    X_train_f = X_train_f.reshape(X_train_f.shape[0], -1)
    X_test_f = X_test_f.reshape(X_test_f.shape[0], -1)

    rf_model_conv = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_model_conv.fit(X_train_f, y_train_labels)

    preds_conv = rf_model_conv.predict(X_test_f)
    acc = accuracy_score(y_test_labels, preds_conv)
    print(f"✅ Accuracy using Conv Layer {i+1}: {acc:.4f}")
    layer_accuracies.append(acc)

    if i == len(feature_extractors_conv) - 1:
        last_conv_preds = preds_conv
        last_conv_idx = i + 1

# Plot RF Accuracy vs. CNN Layer Features
plt.figure(figsize=(10, 4))
plt.plot(range(1, len(layer_accuracies) + 1), layer_accuracies, marker='o')
plt.xticks(range(1, len(layer_accuracies) + 1))
plt.xlabel("Conv2D Layer Number")
plt.ylabel("Accuracy")
plt.title("Random Forest Accuracy vs. CNN Layer Features")
plt.grid(True)
plt.show()

# Confusion matrix for the last convolutional layer's features
if last_conv_preds is not None:
    cm_conv = confusion_matrix(y_test_labels, last_conv_preds)
    disp_conv = ConfusionMatrixDisplay(confusion_matrix=cm_conv, display_labels=unique_labels)
    disp_conv.plot(cmap="Blues", values_format='d')
    plt.title(f"Confusion Matrix - RF on Features from Conv Layer {last_conv_idx}")
    plt.show()

# --- 6. Feature Extraction with Random Forest on Last Dense Layer --- #
feature_layer = None
for layer in reversed(cnn_model.layers):
    if isinstance(layer, Dense) and layer.activation.__name__ != 'softmax': # Get last dense layer before softmax
        feature_layer = layer
        break

if feature_layer is None:
    print("Error: No suitable dense layer found for feature extraction.")
else:
    feature_extractor_dense = Model(inputs=cnn_model.input, outputs=feature_layer.output)

    X_train_features_dense = feature_extractor_dense.predict(X_train)
    X_test_features_dense = feature_extractor_dense.predict(X_test)

    y_train_rf = np.argmax(y_train, axis=1)
    y_test_rf = np.argmax(y_test, axis=1)

    rf_model_dense = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_model_dense.fit(X_train_features_dense, y_train_rf)

    y_pred_rf_dense = rf_model_dense.predict(X_test_features_dense)

    print(f"\n--- Evaluation for Random Forest on Last Dense Layer Features ---")
    accuracy_dense = accuracy_score(y_test_rf, y_pred_rf_dense)
    print(f"Random Forest Accuracy: {accuracy_dense:.4f}")

    cm_dense = confusion_matrix(y_test_rf, y_pred_rf_dense)
    sensitivities = []
    specificities = []

    for i in range(len(cm_dense)):
        TP = cm_dense[i, i]
        FN = np.sum(cm_dense[i, :]) - TP
        FP = np.sum(cm_dense[:, i]) - TP
        TN = np.sum(cm_dense) - (TP + FN + FP)

        sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0
        specificity = TN / (TN + FP) if (TN + FP) > 0 else 0

        sensitivities.append(sensitivity)
        specificities.append(specificity)

    for idx, (sens, spec) in enumerate(zip(sensitivities, specificities)):
        print(f"Class {idx}: Sensitivity = {sens:.4f}, Specificity = {spec:.4f}")

    print("\nClassification Report:")
    print(classification_report(y_test_rf, y_pred_rf_dense, target_names=[str(l) for l in unique_labels]))

    disp_dense = ConfusionMatrixDisplay(confusion_matrix=cm_dense, display_labels=unique_labels)
    disp_dense.plot(cmap="Blues", values_format='d')
    plt.title("Confusion Matrix - RF on Last Dense Layer Features")
    plt.show()

# --- 7. Visualize Filters (Kernels) Learned by CNN --- #
filters = None
for layer in cnn_model.layers:
    if isinstance(layer, Conv2D):
        filters, biases = layer.get_weights()
        break

if filters is not None:
    f_min, f_max = filters.min(), filters.max()
    filters = (filters - f_min) / (f_max - f_min)

    num_filters = filters.shape[3]
    num_channels = filters.shape[2]

    fig, axes = plt.subplots(nrows=4, ncols=8, figsize=(12, 6))
    axes = axes.flatten()

    for i in range(min(num_filters, 32)):
        ax = axes[i]
        if num_channels == 3:
            ax.imshow(filters[:, :, :, i])
        else:
            ax.imshow(filters[:, :, 0, i], cmap='gray')
        ax.axis("off")

    plt.suptitle("Filters from First Conv2D Layer", fontsize=14)
    plt.tight_layout()
    plt.show()
else:
    print("No convolutional filters found to visualize.")


# --- 8. Visualize Feature Maps (Activations) --- #
sample_image_vis = np.expand_dims(X_test[0], axis=0)

layer_outputs_vis = [layer.output for layer in cnn_model.layers if isinstance(layer, Conv2D)]
visualization_model = Model(inputs=cnn_model.input, outputs=layer_outputs_vis)

feature_maps_vis = visualization_model.predict(sample_image_vis)

fig, axes = plt.subplots(nrows=4, ncols=8, figsize=(12, 6))
axes = axes.flatten()

for i in range(min(feature_maps_vis[0].shape[-1], 32)):
    ax = axes[i]
    ax.imshow(feature_maps_vis[0][0, :, :, i], cmap="viridis")
    ax.axis("off")

plt.suptitle(f'Feature Maps from Layer: {conv_layers[0].name}', fontsize=14)
plt.tight_layout()
plt.show()

# --- 9. Project Features Using PCA (from last dense layer features) --- #
if feature_layer is not None:
    pca = PCA(n_components=2)
    X_train_pca = pca.fit_transform(X_train_features_dense)

    plt.figure(figsize=(8, 6))
    sns.scatterplot(x=X_train_pca[:, 0], y=X_train_pca[:, 1], hue=y_train_rf, palette="viridis", legend='full')
    plt.xlabel("Principal Component 1")
    plt.ylabel("Principal Component 2")
    plt.title("PCA Visualization of Extracted Features (from last dense layer)")
    plt.show()
else:
    print("Cannot perform PCA as last dense layer features were not extracted.")

# --- 10. Generate Class Activation Maps (CAMs) --- #
img_for_cam = X_test[0]
img_for_cam_expanded = np.expand_dims(img_for_cam, axis=0)

preds_cam = cnn_model.predict(img_for_cam_expanded)
pred_class_cam = np.argmax(preds_cam[0])

last_conv_layer_cam = None
for layer in reversed(cnn_model.layers):
    if isinstance(layer, Conv2D):
        last_conv_layer_cam = layer
        break

if last_conv_layer_cam is None:
    print("Error: No convolutional layers found in the model for CAM!")
else:
    grad_model_cam = Model(inputs=cnn_model.input, outputs=[last_conv_layer_cam.output, cnn_model.output])
    with tf.GradientTape() as tape:
        conv_output_cam, predictions_cam = grad_model_cam(img_for_cam_expanded)
        loss_cam = predictions_cam[:, pred_class_cam]

    grads_cam = tape.gradient(loss_cam, conv_output_cam)
    pooled_grads_cam = K.mean(grads_cam, axis=(0, 1, 2))

    heatmap_cam = np.mean(conv_output_cam[0], axis=-1)
    heatmap_cam = np.maximum(heatmap_cam, 0)
    if np.max(heatmap_cam) > 0: # Avoid division by zero
        heatmap_cam /= np.max(heatmap_cam)

    img_display_cam = (img_for_cam * 255).astype(np.uint8)
    img_display_cam = cv2.resize(img_display_cam, (IMG_HEIGHT, IMG_WIDTH))

    heatmap_cam = cv2.resize(heatmap_cam, (img_display_cam.shape[1], img_display_cam.shape[0]))
    heatmap_cam = np.uint8(255 * heatmap_cam)
    heatmap_cam = cv2.applyColorMap(heatmap_cam, cv2.COLORMAP_JET)

    superimposed_img_cam = cv2.addWeighted(img_display_cam, 0.6, heatmap_cam, 0.4, 0)

    plt.figure(figsize=(6, 6))
    plt.imshow(cv2.cvtColor(superimposed_img_cam, cv2.COLOR_BGR2RGB))
    plt.title(f"CAM for Predicted Class: {pred_class_cam}")
    plt.axis("off")
    plt.show()
